https://github.com/thisbejim/Pyrebase?tab=readme-ov-file#authentication

In [1]:
import pyrebase
import json

with open("auth.json") as f:
    config = json.load(f)

In [2]:
firebase = pyrebase.initialize_app(config)
db = firebase.database()

In [7]:
# 저장
id = '1234'
data = {"history" : ""}
db.child("User").child(id).set(data)

{'history': ''}

In [8]:
# 수정
id = '1234'
data = {"history" : "언뇽"}
db.child("User").child(id).update(data)

{'history': '언뇽'}

In [9]:
# 조회
a = db.child("User").child(id).get()
a

In [12]:
a.val()['history']

'언뇽'

In [13]:
# 저장
id = 'rag123'
data = {"history" : ""}
db.child("User").child(id).set(data)

{'history': ''}

In [24]:
db.child("User").child('dfad').get().val() is None

True

# 챗봇 합치기

In [25]:
def get_session_history(session_ids):
    print(f"[대화 세션ID]: {session_ids}")
    if db.child("User").child(session_ids).get().val() is None:  # 세션 ID가 data에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 data에 저장
        data = {"history" : ""}
        db.child("User").child(session_ids).set(data) 
    return db.child("User").child(session_ids).get().val()['history'] # 해당 세션 ID에 대한 세션 기록 반환

In [26]:
get_session_history('1234')

[대화 세션ID]: 1234


'언뇽'

In [27]:
get_session_history('dfad')

[대화 세션ID]: dfad


''

In [28]:
db.child("User").get().val()

OrderedDict([('1234', {'history': '언뇽'}),
             ('dfad', {'history': ''}),
             ('rag123', {'history': ''})])

In [29]:
from langchain.prompts import HumanMessagePromptTemplate, ChatPromptTemplate
from langchain.schema import SystemMessage
from langchain.chat_models import ChatOpenAI
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()


# page_content만 저장
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 모델
chatgpt = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature = 0.3
)

# 챗 메세지
chat_messages = [
    SystemMessage(content='당신은 해양 관련 지식을 가지고 있는 사람입니다. 질문에 대해 100자 내외로 말해주고, 해양과 관련된 이야기가 아니면 정중하게 거절해주세요.'),
    HumanMessagePromptTemplate.from_template('{history}'),
    HumanMessagePromptTemplate.from_template('Context: {context}\nQuestion: {ques}')
]

# 챗 프롬프트
chat_prompt = ChatPromptTemplate.from_messages(chat_messages)

vectorstore = Chroma(embedding_function=OpenAIEmbeddings(), persist_directory='C:/project4/chat/testDB')
retriever = vectorstore.as_retriever(search_kwargs={"k":3})


while True:
    id = input('id')
    history = get_session_history(id)
    try:
        q = input('ques')
        
        context = format_docs(retriever.invoke(history+q))
        
        result = chat_prompt.invoke({
            'history': history,
            'context': context,
            'ques': q
        })

        ans = chatgpt.invoke(result).content
        print(ans)
        
        # db 수정
        data = {"history" : history+f'Human: {q}\nAI: {ans}\n'}
        db.child("User").child(id).update(data)

    except:
        print(db.child("User").get().val())
        break

C:\Users\Admin\AppData\Local\Temp\ipykernel_27428\821416107.py:16: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chatgpt = ChatOpenAI(


[대화 세션ID]: 1234
해양쓰레기는 육상 기인과 해상 기인으로 나뉩니다. 육상 기인 쓰레기에는 플라스틱, 유리병, 비닐봉지, 스티로폼 용기, 담배 꽁초가 포함됩니다. 해상 기인 쓰레기에는 플라스틱, 스티로폼 부표, 낚시 용품, 어구 등이 있습니다.
[대화 세션ID]: new_id
해양쓰레기의 원인은 주로 육상에서 발생한 폐기물의 유입과 해상에서의 투기입니다. 플라스틱, 비닐봉지, 담배 꽁초 등이 육상 기인 쓰레기로, 어구와 낚시 용품 등은 해상 기인 쓰레기로 분류됩니다.


KeyboardInterrupt: Interrupted by user